# Object Flaw Detector

The object flaw detector is a simple reference implementation example that uses OpenCV for detecting anomalies in objects on a conveyer belt. The key features demonstrated in this sample are

* Counting the number of objects
* Color detection
* Crack detection 
* Orientation detection
* Laber reader 

More information as well as the original source code can be found on GitHub:

https://github.com/intel-iot-devkit/reference-implementation/tree/master/object-flaw-detector

The implementation here (see <a href="flawdetector.py">flawdetector.py</a>) has been slightly modified to work in the IoT DevCloud environment.

### Demo

Select the cell below containing a Python code (to select, click with the mouse or use keyboard arrows). Then run the cell by pressing **Shift+Enter** or **CTRL+Enter** or by clicking the Run button in the toolbar.

A job is submitted to an edge compute nodes with Intel® Core CPU and Intel® Xeon CPU.
After the inference is completed, the output videos are appropriately stored in the /results/[device] directory, which can then be viewed within the Jupyter Notebook instance.

**Note**: The close button on the video will stop the video, but the application will continue to run. To stop the anomaly detection application itself, interrupt the kernel. You can do this by going in the main menu to Kernel -> Interrupt.

## Set Up

### Import dependencies

Run the below cell to import Python dependencies needed for displaying the results in this notebook
(tip: select the cell and use **Ctrl+enter** to run the cell)

In [ ]:
from pathlib import Path
import sys
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *

## Input Video

Run the following cell to create a symlink and view the input video.

In [ ]:
!ln -sf /data/reference-sample-data/flaw-detector-python/object-flaw-detector.mp4 
videoHTML('Flaw detector video', ['object-flaw-detector.mp4'])

## Create a Job File

The code up to this point has been run within the Jupyter Notebook instance running on a development node based on an Intel® Xeon® Scalable Processor, where the Notebook is allocated a single core. To run inference on the entire video, we need more compute power. We will run the workload on several DevCloud's edge compute nodes. We will send work to the edge compute nodes by submitting jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

To pass the specific variables to the Python code, we will use following arguments:

* `-f`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;location of the input video
* `-d`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output directory

In [ ]:
%%writefile flaw_detector.sh

cd $PBS_O_WORKDIR

VIDEO=$1
OUTPUT_DIR=$2

mkdir -p $OUTPUT_DIR

python3 flawdetector.py -f ${VIDEO} \
                           -d ${OUTPUT_DIR}

In [ ]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

Here, the properties describe the node, and number on the left is the number of available nodes of that architecture.

## Job queue submission

Each of the cells below will submit a job to different edge compute nodes.
The output of the cell is the `JobID` of your job, which you can use to track progress of a job.

**Note** You can submit all jobs at once or follow one at a time. 

After submission, they will go into a queue and run as soon as the requested compute resources become available. 
(tip: **shift+enter** will run the cell and automatically move you to the next cell. So you can hit **shift+enter** multiple times to quickly run multiple cells)

#### Submitting to an edge compute node with an Intel® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core™ i5-6500TE processor</a>. The inference workload will run on the CPU

In [ ]:
#Submit job to the queue
job_id_core = !qsub flaw_detector.sh -l nodes=1:idc001skl:tank-870:i5-6500te -F "/data/reference-sample-data/flaw-detector-python/object-flaw-detector.mp4 results/core" -N flaw_det_core
print(job_id_core[0])
if job_id_core:
    progressIndicator('results/core', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)

#### Submitting to an edge compute node with Intel® Xeon® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz-">Intel® 
    Xeon® Processor E3-1268L v5</a>. The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon = !qsub flaw_detector.sh  -l nodes=1:idc007xv5:e3-1268l-v5 -F "/data/reference-sample-data/flaw-detector-python/object-flaw-detector.mp4 results/xeon" -N flaw_det_xeon
print(job_id_xeon[0]) 
#Progress indicators
if job_id_xeon:
    progressIndicator('results/xeon/', 'i_progress_'+job_id_xeon[0]+'.txt', "Inference", 0, 100)

### Check if the jobs are done

To check on the jobs that were submitted, use the `qstat` command.

We have created a custom Jupyter widget  to get live qstat update.
Run the following cell to bring it up.

In [ ]:
liveQstat()

You should see the jobs you have submitted (referenced by `Job ID` that gets displayed right after you submit the job).
There should also be an extra job in the queue "jupyterhub": this job runs your current Jupyter Notebook session.

The 'S' column shows the current status. 
- If it is in Q state, it is in the queue waiting for available resources. 
- If it is in R state, it is running. 
- If the job is no longer listed, it means it is completed.

**Note**: Time spent in the queue depends on the number of users accessing the edge nodes. Once these jobs begin to run, they should take from 1 to 5 minutes to complete.

## View Results

Once the jobs are completed, the queue system outputs the stdout and stderr streams of each job into files with names of the form

`flaw_det_{type}.o{JobID}`

`flaw_det_{type}.e{JobID}`

(here, flaw_det_{type} corresponds to the `-N` option of qsub).

However, for this case, we may be more interested in the output video files. They are stored in mp4 format inside the `results/[device]` directory.
We wrote a short utility script that will display these videos within the notebook.
Run the cells below to display them.
See `demoutils.py` if you are interested in understanding further on how the results are displayed in notebook.

In [ ]:
videoHTML('IEI Tank (Intel Core CPU)', 
         ['results/core/inference_output.mp4'],'results/core/stats.txt')

In [ ]:
videoHTML('IEI Tank (Intel Xeon CPU)', 
         ['results/xeon/inference_output.mp4'],'results/xeon/stats.txt')